# AGB model Jan

## Imports

In [1]:
import vtk
import meshio
import numpy          as np
import magritte.setup as setup
import magritte.core  as magritte

from tqdm                   import tqdm
from astropy                import constants
from vtk.util.numpy_support import vtk_to_numpy
from scipy.spatial          import Delaunay

## Dowload Jan's model from Frederik's nextcloud account (~400MB, so might take a while).

In [2]:
!wget -O "model.vtu" https://owncloud.ster.kuleuven.be/index.php/s/mqtDyDSMPm2TjmG/download

--2021-03-23 14:19:59--  https://owncloud.ster.kuleuven.be/index.php/s/mqtDyDSMPm2TjmG/download
Resolving owncloud.ster.kuleuven.be (owncloud.ster.kuleuven.be)... 134.58.130.75
Connecting to owncloud.ster.kuleuven.be (owncloud.ster.kuleuven.be)|134.58.130.75|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 415691447 (396M) [application/octet-stream]
Saving to: ‘model.vtu’

model.vtu           100%[===================>] 396.43M  21.3MB/s    in 23s     

2021-03-23 14:20:23 (17.1 MB/s) - ‘model.vtu’ saved [415691447/415691447]



In [3]:
vtu_file   = 'model.vtu'
model_file = 'model.hdf5'
#lamda_file = '../data/co.txt'
lamda_file = '../data/test.txt'

## Extract the required data from the AMRVAC vtu file.

In [4]:
# Create a vtk reader to read the AMRVAC file
reader = vtk.vtkXMLUnstructuredGridReader()
reader.SetFileName(vtu_file)
reader.Update()

# Read the model
grid = reader.GetOutput()

# Set model parameters
dimension = 3
npoints   = grid.GetNumberOfCells()
nrays     = 12
nspecs    = 5
nlspecs   = 1
nquads    = 1

# Extract cell data
cellData = grid.GetCellData()
for i in tqdm(range(cellData.GetNumberOfArrays())):
    array = cellData.GetArray(i)
    if (array.GetName() == 'rho'):
        rho = vtk_to_numpy(array)
    if (array.GetName() == 'temperature'):
        tmp = vtk_to_numpy(array)
    if (array.GetName() == 'v1'):
        v_x = vtk_to_numpy(array)
    if (array.GetName() == 'v2'):
        v_y = vtk_to_numpy(array)
    if (array.GetName() == 'v3'):
        v_z = vtk_to_numpy(array)

# Convert rho (total density) to abundances
nH2 = rho * 1.0e+6 * constants.N_A.si.value / 2.02
nCO = nH2 * 1.0e-4

# Convert to fractions of the speed of light
velocity = np.array((v_x, v_y, v_z)).transpose() / constants.c.cgs.value

# Defince turbulence at 150 m/s
trb = (150.0/constants.c.si.value)**2 * np.ones(grid.GetNumberOfCells())

# Extract cell centres as Magritte points
centres = []
for c in tqdm(range(grid.GetNumberOfCells())):
    cell = grid.GetCell(c)
    centre = np.zeros(3)
    for i in range(8):
        centre = centre + np.array(cell.GetPoints().GetPoint(i))
    centre = 0.125 * centre
    centres.append(centre)
centres  = np.array(centres)
centres *= 1.0e-2   # convert [cm] to [m]
position = centres

# we assume that the geometry to be a cube centred around the origin.
# we assume that there is (at least) one face of the cube that has not been refined
# (or that is completely covered by the coarsest elements that can be found on the boundary.).
bound = 0.999*np.min(np.max(np.abs(centres), axis=0))
x_max = y_max = z_max =  bound
x_min = y_min = z_min = -bound

# Extract the (artificial) boundary
boundary = []
for i, (x,y,z) in tqdm(enumerate(centres)):
    if not ((x_min < x < x_max) and
            (y_min < y < y_max) and
            (z_min < z < z_max)    ):
        boundary.append(i)
boundary = np.array(boundary)

# Extract Delaunay vertices (= Voronoi neighbors)
delaunay = Delaunay(centres)
(indptr, indices) = delaunay.vertex_neighbor_vertices
neighbors = [indices[indptr[k]:indptr[k+1]] for k in range(npoints)]
nbs       = [n for sublist in neighbors for n in sublist]
n_nbs     = [len(sublist) for sublist in neighbors]

# Convenience arrays
zeros = np.zeros(npoints)
ones  = np.ones (npoints)

100%|██████████| 297984/297984 [00:06<00:00, 45405.31it/s]
297984it [00:00, 708687.03it/s]


In [4]:
sum(n_nbs)==len(nbs)

True

## Create a Magritte model from the data.

In [5]:
model = magritte.Model ()
model.parameters.set_spherical_symmetry(False)
model.parameters.set_pop_prec          (1.0e-6)
model.parameters.set_model_name        (model_file)
model.parameters.set_dimension         (dimension)
model.parameters.set_npoints           (npoints)
model.parameters.set_nrays             (nrays)
model.parameters.set_nspecs            (nspecs)
model.parameters.set_nlspecs           (nlspecs)
model.parameters.set_nquads            (nquads)

model.geometry.points.position.set(position)
model.geometry.points.velocity.set(velocity)

#model.geometry.points.  neighbors.set(  nbs)
#model.geometry.points.n_neighbors.set(n_nbs)
model.geometry.points.multiscale.set_all_neighbors(n_nbs,nbs)

model.chemistry.species.abundance = np.array((zeros, nCO, nH2, zeros, ones)).T
# model.chemistry.species.symbol    = ['dummy0', 'CO', 'H2', 'e-', 'dummy1']
model.chemistry.species.symbol    = ['dummy0', 'test', 'H2', 'e-', 'dummy1']

model.thermodynamics.temperature.gas  .set(tmp)
model.thermodynamics.turbulence.vturb2.set(trb)

model.parameters.set_nboundary(boundary.shape[0])
model.geometry.boundary.boundary2point.set(boundary)

model = setup.set_boundary_condition_CMB  (model)
model = setup.set_uniform_rays            (model)
# model = setup.set_linedata_from_LAMDA_file(model, lamda_file, {'considered transitions': [0, 1]})
model = setup.set_linedata_from_LAMDA_file(model, lamda_file)
model = setup.set_quadrature              (model)

model.write()

In [6]:
model.parameters.npoints()

297984

## Run the Magritte model.

In [7]:
model = magritte.Model (model_file)

model.compute_spectral_discretisation ()
model.compute_inverse_line_widths     ()
model.compute_LTE_level_populations   ()

0